In [2]:
from dotenv import find_dotenv, load_dotenv

load_dotenv((find_dotenv()))

True

🟢 Part A — LCEL (|) Chain
Build a tone-safe reply generator using: prompt | llm | StrOutputParser()


In [22]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate, ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda

In [23]:
llm = ChatOpenAI(model='gpt-4o-mini')

In [24]:
system_message = SystemMessagePromptTemplate.from_template("""You are a customer support assistant for an e-commerce company.
Tone rules:
- Be polite, calm, professional.
- Keep it concise (<= 120 words).
- Must include: apology + next step + timeline.
- Do NOT promise refund unless the policy explicitly allows it.
- No greetings like 'Dear customer'. Use a friendly neutral tone.""")

human_message = HumanMessagePromptTemplate.from_template("""Customer message:
{customer_message}

Company policy snippet:
{company_policy_snippet}

Write the best reply.""")

message = [system_message, human_message]
reply_prompt = ChatPromptTemplate(message)

sample = {
    "customer_message": "My parcel is late again. This is the 3rd time. I want my money back.",
    "company_policy_snippet": "Refund only if delivery delay > 10 days beyond promised date; otherwise offer expedited replacement."
}

chain1 = reply_prompt | llm | StrOutputParser()

reply = chain1.invoke(sample)


In [25]:
print(reply)

I apologize for the inconvenience caused by the delay of your parcel. Unfortunately, since the delay has not exceeded 10 days beyond the promised date, I'm unable to process a refund. However, I can offer you an expedited replacement to ensure you receive your items as quickly as possible. 

Please confirm if you'd like to proceed with the replacement, and I will initiate the process immediately. You can expect the replacement to ship within 1-3 business days. Thank you for your understanding.


🟡 Part B — Sequential Chain: Classify → Reply
We build:

Classifier using PromptTemplate
Reply writer using Chat prompt templates
Connect them so Step-1 output becomes an input to Step-2

In [31]:
# From the customer report Identify the class of Issue from the list:
# Once the issue class if identified, write reply to the whole message 

issue_list = [
    'Delivery Delayed',
    'Damaged Item',
    'Wrong Message',
    'Payment failure',
    'Account/ Logic issue',
    'Others'
]

classifier_prompt = PromptTemplate.from_template('''
                    Classifiy the customer issue Exactly into one of the labels :  {issue_list}  
                                                              Customer Message: {customer_message}
                                                              Return only return the label.''')
classifier_chain = classifier_prompt | llm | StrOutputParser()

reply_prompt = SystemMessagePromptTemplate.from_template(    """You are a customer support assistant.
Rules:
- Professional, concise (<= 120 words)
- Must include: apology + next step + timeline
- Follow policy and do not promise refund unless allowed
- Use issue label to tailor the response.""")


reply_human = HumanMessagePromptTemplate.from_template('''
Issue Lable: {issue_label}, 
Customer Message: {customer_message}, 
Policy Statement: {company_policy_statement}, 
Write the best reply''')


reply_prompt = ChatPromptTemplate.from_messages([reply_prompt, reply_human])

reply2_chain = reply_prompt | llm | StrOutputParser()


# --- Sequential wiring ---
# Step 1 creates issue_label, Step 2 uses it.
sequential_chain = (
    RunnablePassthrough.assign(
        issue_label=(RunnableLambda(lambda x: {"labels": ", ".join(issue_list), **x}) | classifier_chain)
    )
    | reply2_chain
)

policy = "Refund only if delivery delay > 10 days beyond promised date; otherwise offer expedited replacement."
msgs = [
    "My order 12345 is delayed again. Very frustrating.",
    "Received a broken screen on my phone.",
    "You charged me twice!"
]

for m in msgs:
    out = sequential_chain.invoke({"customer_message": m, "company_policy_statement": policy, "issue_list": ", ".join(issue_list)})
    print("\n---\n", m)
    print(out)


---
 My order 12345 is delayed again. Very frustrating.
Dear Customer,

I sincerely apologize for the delay with your order 12345. I understand how frustrating this situation can be. To address this, we can offer you an expedited replacement, which should arrive within 3-5 business days. 

Please confirm if you'd like to proceed with this option, and I'll initiate the process immediately.

Thank you for your understanding.

Best regards,  
[Your Name]  
Customer Support Team 

---
 Received a broken screen on my phone.
Dear Customer,

I apologize for receiving a damaged screen on your phone. We understand how frustrating this can be. To resolve this, we will expedite a replacement for you. 

Please allow 3-5 business days for the new device to ship. If you have any further questions or need assistance, feel free to reach out.

Thank you for your understanding.

Best regards,  
[Your Name]  
Customer Support

---
 You charged me twice!
Dear Customer,

I apologize for the inconvenience 